In [ ]:
task = "ner" # Should be one of "ner", "pos" or "chunk"
model_checkpoint = "bert-base-cased"
batch_size = 16
from datasets import load_dataset, load_metric
from datasets import DatasetDict

#todo need to change and train on full dataset in final version
jnlpba = load_dataset('jnlpba', split=['train[:10]', "validation[:10]"])

datasets = DatasetDict({"train": jnlpba[0], "validation": jnlpba[1]})

label_list = datasets["train"].features[f"{task}_tags"].feature.names

import random
from datasets import Dataset

def weak(lst):
    random_number = 0
    item = random_number
    item_actual = 0
    item_set = 0
    while item_actual == item_set:
        random_number = random.randint(0, len(lst)-1)
        random_number_item = random.randint(0, len(label_list)-1)
        item_set = random_number_item
        item_actual = lst[random_number]
    new_lst = lst.copy()
    new_lst[random_number] = item_set
    return new_lst
df = datasets["train"].to_pandas()
df["ner_tags"] = df["ner_tags"].apply(weak)

dataset_train = Dataset.from_pandas(df)

datasets = DatasetDict({"train": dataset_train, "validation": jnlpba[1]})

#i think the problem is that the new dataset which i created is not the same format like the dataset before
#can check that in starting training again without weak() method. if this works than I can try to exchange only the rows'''



from transformers import AutoTokenizer
    
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)


import transformers
assert isinstance(tokenizer, transformers.PreTrainedTokenizerFast)



label_all_tokens = True

def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples[f"{task}_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            # Special tokens have a word id that is None. We set the label to -100 so they are automatically
            # ignored in the loss function.
            if word_idx is None:
                label_ids.append(-100)
            # We set the label for the first token of each word.
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            # For the other tokens in a word, we set the label to either the current label or -100, depending on
            # the label_all_tokens flag.
            else:
                label_ids.append(label[word_idx] if label_all_tokens else -100)
            previous_word_idx = word_idx

        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs



tokenized_datasets = datasets.map(tokenize_and_align_labels, batched=True)


from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer

model = AutoModelForTokenClassification.from_pretrained(model_checkpoint, num_labels=len(label_list))

model_name = model_checkpoint.split("/")[-1]
args = TrainingArguments(
    f"{model_name}-finetuned-{task}-weak-labelled",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=3,
    weight_decay=0.01,
)


from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer)

metric = load_metric("seqeval")

import numpy as np

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

trainer = Trainer(
model,
args=args,
train_dataset=tokenized_datasets["train"],
eval_dataset=tokenized_datasets["validation"],
data_collator=data_collator,
tokenizer=tokenizer,
compute_metrics=compute_metrics
)
trainer.train()
trainer.evaluate()

predictions, labels, _ = trainer.predict(tokenized_datasets["validation"])
predictions = np.argmax(predictions, axis=2)

# Remove ignored index (special tokens)
true_predictions = [
    [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]
true_labels = [
    [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]

results = metric.compute(predictions=true_predictions, references=true_labels)
results

### Dataset

![Alt Text](../../images/weak_ner_example.png)

In [1]:
#there are basically three different options 
#1. label is missing 2. wrong label 3. Mising inside
#in each entry where I make the dataset weak I apply one of this methods 

Found cached dataset jnlpba (/Users/maxhager/.cache/huggingface/datasets/jnlpba/jnlpba/1.0.0/3062f220823930cffde7976b694aa67bac3b06c322a02ced92d3761519810ce4)


  0%|          | 0/2 [00:00<?, ?it/s]

In [4]:
from datasets import load_dataset, load_metric
from datasets import DatasetDict, Dataset
jnlpba = load_dataset('jnlpba', split=['train[:10]', "validation[:10]"])
jnlpba = DatasetDict({"train": jnlpba[0], "validation": jnlpba[1]})
#turn jnlpba into dataframe
df = jnlpba["train"].to_pandas()

print(df)


Found cached dataset jnlpba (/Users/maxhager/.cache/huggingface/datasets/jnlpba/jnlpba/1.0.0/3062f220823930cffde7976b694aa67bac3b06c322a02ced92d3761519810ce4)


  0%|          | 0/2 [00:00<?, ?it/s]

   id                                             tokens  \
0   1  [IL-2, gene, expression, and, NF-kappa, B, act...   
1   2  [Activation, of, the, CD28, surface, receptor,...   
2   3  [In, primary, T, lymphocytes, we, show, that, ...   
3   4  [Delineation, of, the, CD28, signaling, cascad...   
4   5  [Our, data, suggest, that, lipoxygenase, metab...   
5   6  [These, findings, should, be, useful, for, the...   
6   7  [The, peri-kappa, B, site, mediates, human, im...   
7   8  [Human, immunodeficiency, virus, type, 2, (, H...   
8   9  [HIV-1, and, HIV-2, display, significant, diff...   
9  10  [Consistent, with, these, differences, ,, we, ...   

                                            ner_tags  
0  [1, 2, 0, 0, 9, 10, 0, 0, 9, 0, 0, 0, 0, 0, 9, 0]  
1  [0, 0, 0, 9, 10, 10, 0, 0, 0, 0, 0, 0, 0, 0, 0...  
2  [0, 7, 8, 8, 0, 0, 0, 9, 0, 0, 0, 0, 0, 0, 0, ...  
3  [0, 0, 0, 9, 0, 0, 0, 0, 0, 0, 9, 10, 10, 0, 0...  
4  [0, 0, 0, 0, 9, 10, 0, 0, 0, 0, 0, 0, 9, 0, 0,...  
5  [0, 0,

In [19]:
from datasets import load_dataset, load_metric
from datasets import DatasetDict, Dataset
import random
import pandas as pd
random.seed(42)


jnlpba = load_dataset('jnlpba', split=['train[:3000]', "validation[:100]"])
jnlpba = DatasetDict({"train": jnlpba[0], "validation": jnlpba[1]})


class JnlpbDataset():
    def __init__(self, dataset, portion, type_path):
        self.dataset = dataset[type_path]
        self.portion = portion
        self.merge()
        self.apply()
        
    def map_tags(self, row):
        mapping = {
            0: "O",
            1: "B-DNA",
            2: "I-DNA",
            3: "B-RNA",
            4: "I-RNA",
            5: "B-cell_line",
            6: "I-cell_line",
            7: "B-cell_type",
            8: "I-cell_type",
            9: "B-protein",
            10: "I-protein"
        }
        row['ner_tags'] = [[mapping[tag] for tag in row['ner_tags']]][0]
        return row        

    def merge_tags(self, tags, tokens):
      #todo test if this works also in the scenario of having two B- tags side by side
      merged_tags = []
      merged_tokens = []
      i = 0
      while i < len(tags):
          if tags[i].startswith('B-'):
              merged_tag = tags[i][2:]
              merged_token = tokens[i]
              i += 1
              while i < len(tags) and tags[i].startswith('I-'):
                  merged_tag += ' ' + tags[i][2:]
                  merged_token += ' ' + tokens[i]
                  i += 1
              merged_tags.append(merged_tag)
              merged_tokens.append(merged_token)
          else:
              merged_tags.append(tags[i])
              merged_tokens.append(tokens[i])
              i += 1
      for i in range(len(merged_tags)):
        s = merged_tags[i].split()[0]
        s = s[0].upper() + s[1:]
        merged_tags[i] = s
      return merged_tags, merged_tokens

    def merge(self):
      df_train = pd.DataFrame(self.dataset)
      df_train = df_train.apply(self.map_tags, axis=1)
      df_train[['ner_tags', 'tokens']] = df_train.apply(lambda x: self.merge_tags(x['ner_tags'], x['tokens']), axis=1, result_type='expand')
      self.dataset = Dataset.from_pandas(df_train)
        
    def missing(self, row):
        #print(len(self.label_list))
        lst = row["ner_tags"]
        if any(x != 0 for x in lst):
            index = random.choice([i for i, x in enumerate(lst) if x != 0])
            lst[index] = 0
            row["ner_tags"] = lst
            return row
        else:
            return row

    def wrong(self, row, num_tags):
        lst = row["ner_tags"]
        tags = []
        #it would be nice to have a debugger who can display me the values of the vars
        for i in range(1,num_tags):
            tags.append(i)
        if any(x != 0 for x in lst):    
            indices = [i for i, x in enumerate(lst) if x != 0]
            random_index = random.choice(indices)
            current_value = lst[random_index]
            random_number = random.choice([x for x in [1, 2, 3, 4, 5] if x != current_value])
            lst[random_index] = random_number
            row["ner_tags"] = lst
            return row
        else:
            return row
                        
    def uncomplete(self):
        #todo should i implement this
        pass

    def apply(self):
        shuffled_data = self.dataset.shuffle()
        num_portion = int(len(self.dataset) * self.portion / 100)
        df = self.dataset.to_pandas() 
        tags = [tag for row in df['ner_tags'] for tag in row]
        unique_tags = set(tags)
        mapping = {}

        for index, item in enumerate(unique_tags):
            mapping[item] = index        
        #iter over df ner tags column and set each string to number from mapping
        df['ner_tags'] = [[mapping[tag] for tag in tags] for tags in df['ner_tags']]    
        
        for i in range(num_portion):
            random_number = random.randint(1, 2)
            if random_number == 1:
                new_row = self.missing(df.iloc[i])
                df.iloc[i] = new_row
            elif random_number == 2:
                num_tags = len(unique_tags)
                new_row = self.wrong(df.iloc[i], num_tags)
                df.iloc[i] = new_row
            '''else:
                self.uncomplete()'''
        self.dataset = Dataset.from_pandas(df)
        
    def get_dataset(self):
        return self.dataset
        
input_dataset = JnlpbDataset(dataset=jnlpba, portion=20, type_path='train')
dataset = input_dataset.get_dataset()

Found cached dataset jnlpba (/Users/maxhager/.cache/huggingface/datasets/jnlpba/jnlpba/1.0.0/3062f220823930cffde7976b694aa67bac3b06c322a02ced92d3761519810ce4)


  0%|          | 0/2 [00:00<?, ?it/s]

/var/folders/r9/30wn6mvs2md_xllcvl5lpn0m0000gn/T/ipykernel_34595/2487113789.py:74: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  row["ner_tags"] = lst
/var/folders/r9/30wn6mvs2md_xllcvl5lpn0m0000gn/T/ipykernel_34595/2487113789.py:91: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  row["ner_tags"] = lst


In [ ]:
from datasets import load_dataset, load_metric
from datasets import DatasetDict

#todo need to change and train on full dataset in final version
jnlpba = load_dataset('jnlpba', split=['train[:10]', "validation[:10]"])

datasets = DatasetDict({"train": jnlpba[0], "validation": jnlpba[1]})

label_list = datasets["train"].features[f"{task}_tags"].feature.names

import random
from datasets import Dataset

def weak(lst):
    random_number = 0
    item = random_number
    item_actual = 0
    item_set = 0
    while item_actual == item_set:
        random_number = random.randint(0, len(lst)-1)
        random_number_item = random.randint(0, len(label_list)-1)
        item_set = random_number_item
        item_actual = lst[random_number]
    new_lst = lst.copy()
    new_lst[random_number] = item_set
    return new_lst
df = datasets["train"].to_pandas()
df["ner_tags"] = df["ner_tags"].apply(weak)

dataset_train = Dataset.from_pandas(df)

datasets = DatasetDict({"train": dataset_train, "validation": jnlpba[1]})